## D2RL Data Process

In [ ]:
root_folder = "../raw_data/D2RL/"
path_list = ["Experiment-2lane_400m_D2RL_2022-09-10"]

In [ ]:
import glob
import json
from tqdm import tqdm
import numpy as np
from multiprocessing import Pool
import random

import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx, array[idx]

x_list = np.linspace(400, 800, num=201)
y_list = np.linspace(40, 48, num=5)

def get_occupied_box_index_from_obs(vehicle_obs):
    x,y = vehicle_obs["position"]
    x_id, near_x = find_nearest(x_list, x)
    y_id, near_y = find_nearest(y_list, y)
    surrounding_index = [[x_id, y_id], [x_id-1, y_id], [x_id-2, y_id]]
    surrounding_index = [[x_id, y_id]]
#     print(nearest_index + surrounding_index)
    return surrounding_index

def calculate_pet_single_vehicle(cav_timestep, time_id_list):
    min_pet = 10
    for veh_time_id in time_id_list:
        time, veh_id = veh_time_id
        if veh_id == "CAV":
            continue
        else:
            min_pet = min(min_pet, abs(float(time)-float(cav_timestep)))
#     print(min_pet)        
    return min_pet

def calculate_position_pet_list(time_id_list):
    min_pet = 10
    if len(time_id_list) == 1:
        return min_pet
    for veh_time_id in time_id_list:
        time, veh_id = veh_time_id
        if veh_id == "CAV":
#             print(time_id_list)
            min_pet_tmp = calculate_pet_single_vehicle(time, time_id_list)
            min_pet = min(min_pet, min_pet_tmp)
#             print(min_pet)
    return min_pet

def get_pet_list_from_one_json(json_path):
    try:
#     if 1:
        pet_list = []
#         print(json_path)
        with open(json_path, "r") as json_file:
            json_dict = json.load(json_file)
        weight = json_dict["weight_episode"]
        av_obs = json_dict["av_obs"]
        pet_dict = {}
        for timestep in av_obs:
            for key in av_obs[timestep]:
                tmp_vehicle_obs = av_obs[timestep][key]
                if tmp_vehicle_obs is not None:
                    occupied_index_list = get_occupied_box_index_from_obs(tmp_vehicle_obs)
                    for occupied_index in occupied_index_list:
#                         print(str(occupied_index))
                        if str(occupied_index) in pet_dict:
                            pet_dict[str(occupied_index)].append([timestep, tmp_vehicle_obs["veh_id"]])
                        else:
                            pet_dict[str(occupied_index)] = [[timestep, tmp_vehicle_obs["veh_id"]]]
        for key in pet_dict:
            pet_tmp = calculate_position_pet_list(pet_dict[key])
            if pet_tmp < 10:
                pet_list.append(pet_tmp)
        return pet_list, weight
    except:
        return [], 0

from tqdm import tqdm
def get_pet_from_experiments(experiment_path):
    crash_folder_path = root_folder + experiment_path + "/crash/"
    safe_folder_path = root_folder + experiment_path + "/tested_and_safe/"
    crash_json_list = glob.glob(crash_folder_path + "*.json")
    safe_json_list = glob.glob(safe_folder_path + "*.json")
    
    pet_list_total, weight_list_total = [], []
    pool = Pool(10)
    total_near_miss_json_list = crash_json_list+safe_json_list
    for res in tqdm(pool.imap_unordered(get_pet_list_from_one_json, total_near_miss_json_list)):
#     for crash_json in tqdm(crash_json_list):
        pet_list, weight = res
        pet_list_total.extend(pet_list)
        weight_list_total.extend([weight]*len(pet_list))
    return pet_list_total, weight_list_total
        
def get_pet_from_experiments_NDE(experiment_path):
    crash_folder_path = root_folder + experiment_path + "/crash/"
    safe_folder_path = root_folder + experiment_path + "/tested_and_safe/"
    crash_json_list = glob.glob(crash_folder_path + "*.json")
    safe_json_list = glob.glob(safe_folder_path + "*.json")
    
    pet_list_total, weight_list_total = [], []
    pool = Pool(10)
    total_near_miss_json_list = crash_json_list+safe_json_list
    for res in tqdm(pool.imap_unordered(get_pet_list_from_one_json, total_near_miss_json_list)):
#     for crash_json in tqdm(crash_json_list):
        pet_list, weight = res
        pet_list_total.extend(pet_list)
        weight_list_total.extend([weight]*len(pet_list))
    return pet_list_total, weight_list_total


In [ ]:
pet_list_all_experiments, pet_weight_list_all_experiments = [], []
for experiment_path in path_list:
    pet_list, weight_list = get_pet_from_experiments(experiment_path)
    pet_list_all_experiments.extend(pet_list)
    pet_weight_list_all_experiments.extend(weight_list)
    

140250it [08:20, 280.05it/s]


In [ ]:
import numpy as np
save_folder = "./D2RL/"
np.save(save_folder + "NADE_near_miss_pet.npy", pet_list_all_experiments)
np.save(save_folder + "NADE_near_miss_pet_weight.npy", pet_weight_list_all_experiments)

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(pet_list_all_experiments, weights = pet_weight_list_all_experiments, bins=20)

## NDE Data Process

In [ ]:
root_folder = "../raw_data/NDE/"
path_list = ["Experiment-2lane_400m_NDE_v1_2022-09-09", "Experiment-2lane_400m_NDE_v2_2022-09-09", "Experiment-2lane_400m_NDE_v3_2022-09-09"]

In [ ]:
pet_list_all_experiments, pet_weight_list_all_experiments = [], []
for experiment_path in path_list:
    pet_list, weight_list = get_pet_from_experiments_NDE(experiment_path)
    pet_list_all_experiments.extend(pet_list)
    pet_weight_list_all_experiments.extend(weight_list)
    

13458it [00:55, 243.61it/s]
4657it [00:16, 289.31it/s]
19173it [01:06, 290.09it/s]


In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(pet_list_all_experiments, bins=20)
# # plt.hist(pet_list_all_experiments, weights = pet_weight_list_all_experiments, bins=20)

In [ ]:
import numpy as np
save_folder = "./NDE/"
np.save(save_folder + "NDE_near_miss_pet.npy", pet_list_all_experiments)
np.save(save_folder + "NDE_near_miss_pet_weight.npy", pet_weight_list_all_experiments)